# 実践データ科学入門 2020年度木曜4限

# 第3回 その2 多項式回帰

In [ ]:
%matplotlib inline
#%matplotlib notebook # if necessary to rotate figures in 3D plot
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.patches as patches
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d import art3d
from ipywidgets import interact
from sklearn import linear_model

## 多項式回帰とは

多項式回帰は説明変数 $x$ 1つの場合に，$x$ による単回帰ではなく，$x$, $x^2$, $x^3$ など，$x$ のべき乘の功を回帰変数に用いる手法で，

$$
y = a_0 + a_1 x + a_2 x^2 + \ldots + a_m x^m = \sum_{j=0}^M a_j x^j + \xi
$$

というモデルを当てはめる方法である．多項式回帰モデルは，回帰変数を $x_j = x^j$ とした重回帰モデルであると言える．

データに対して単回帰を用いて直線を当てはめるのが不適切とわかる場合に用いられる．

ここで

- $y$ は目的変数（データとして与えられるもの）
- $x^j \ (j=0, 1, 2, \ldots, M)$ を説明変数と取る（データとして与えられるもの）
- $a_j \ (j=0, 1, 2, \ldots, M)$ は回帰係数（データから求めるもの）
- $\xi$ はノイズ（モデルでは当てはめられないランダムな要因）

以下，特に注意しない限り__データは実数値__とする．

In [ ]:
# 真のパラメータ
A0 = 1.2
A1 = -5.6
A2 = 2.2

# dataset
X = np.arange(0, 3, 0.3)
N = X.size
Y = A0 + A1*X + A2*X**2 +0.5*np.random.randn(N)

In [ ]:
# 回帰直線のプロット
def plot_XY_and_regressionline(a0=0.0, a1=1.0):
    fig = plt.figure()
    ax = plt.axes()
    ax.set_xlabel("X", size=20)
    ax.set_ylabel("Y", size=20)
    ax.set_xticks
    ax.set_ylim(-3, 3)
    ax.scatter(X, Y)
    ax.plot([np.min(X), np.max(X)], [a0+a1*np.min(X), a0+a1*np.max(X)], linewidth=3, color='tab:red')
    ax.set_title('MSE = %f'%(np.sum((Y-a0-a1*X)**2)/Y.size), size=20)
    ax.tick_params(labelsize=12)

In [ ]:
interact(plot_XY_and_regressionline, a0=(-5.0, 5.0, 0.1), a1=(-5.0, 10.0, 0.1))

# 青点がデータ点
# 赤が回帰直線
# MSE = Mean Squared Error = 平均二乗誤差

# 必ずしも真のパラメータのときに平均二乗誤差最小になるわけではないことに注意しよう

直線でのフィッティングでは適切なモデルとは言えない．多項式フィッティングを考えよう．

### 多項式フィッティング

回帰変数2つの重回帰では，データ空間 $(x_1, x_2, y)$ の3次元空間に回帰平面を引き，データから平面までの高さの二乗和が最小になるように選ぶ．

In [ ]:
# 真のパラメータ
A0 = 1.2
A1 = -5.6
A2 = 2.2

# dataset
X = np.arange(0, 3, 0.3)
Y = A0 + A1*X + A2*X**2 +0.5*np.random.randn(X.size)

In [ ]:
# フィッティングに用いる多項式の最大次数
M = 2

# dataset
X_train = np.zeros((X.size, M))

reg = linear_model.LinearRegression()
for j in range(M):
    X_train[:, j] = X**(j+1)

reg.fit(X_train, Y)

A_pred = np.zeros(M+1)
A_pred[0] = reg.intercept_
A_pred[1:M+1] = reg.coef_

XX = np.arange(-0.2, 3, 0.1)
YY = np.ones(XX.size)*A_pred[0]
Y_pred = np.ones(X.size)*A_pred[0]
for j in range(1, M+1):
    YY += A_pred[j] * XX**j
    Y_pred += A_pred[j] * X**j

fig = plt.figure()
ax = plt.axes()
ax.set_xlabel("X", size=20)
ax.set_ylabel("Y", size=20)
ax.set_xticks
ax.set_ylim(-3, 3)
ax.scatter(X, Y, s=80)
ax.plot(XX, YY, linewidth=3, c='tab:orange')
ax.set_title('MSE = %f'%(np.sum((Y-Y_pred)**2)/Y.size), size=20)
ax.tick_params(labelsize=12)

print('A0 = %f'%(reg.intercept_))
for j in range(1, M+1):
    print('A%1d = %f'%(j, reg.coef_[j-1]))

2次関数でうまくフィッティングができている．

しかし，フィッティングに用いる多項式の次数を増やすと MSE をもっと下げることができる．

In [ ]:
# 多項式回帰のプロット
def plot_polynomialregression(M=2):
    # dataset
    X_train = np.zeros((N, M))

    reg = linear_model.LinearRegression()
    for j in range(M):
        X_train[:, j] = X**(j+1)

    reg.fit(X_train, Y)

    A_pred = np.zeros(M+1)
    A_pred[0] = reg.intercept_
    A_pred[1:M+1] = reg.coef_

    XX = np.arange(-0.2, 3, 0.01)
    YY = np.ones(XX.size)*A_pred[0]
    Y_pred = np.ones(X.size)*A_pred[0]
    for j in range(1, M+1):
        YY += A_pred[j] * XX**j
        Y_pred += A_pred[j] * X**j

    fig = plt.figure()
    ax = plt.axes()
    ax.set_xlabel("X", size=20)
    ax.set_ylabel("Y", size=20)
    ax.set_xticks
    ax.set_ylim(-5, 5)
    ax.scatter(X, Y, s=80)
    ax.plot(XX, YY, linewidth=3, c='tab:orange')
    ax.set_title('MSE = %f'%(np.sum((Y-Y_pred)**2)/Y.size), size=20)
    ax.tick_params(labelsize=12)

    #print('A0 = %f'%(reg.intercept_))
    #for j in range(1, M+1):
    #    print('A%1d = %f'%(j, reg.coef_[j-1]))

In [ ]:
interact(plot_polynomialregression, M=(1, 20, 1))

多項式の次数をデータ数と比べて十分高く取ると，全ての点を通るような多項式曲線を描くことができる．この時はデータ点に対する回帰誤差は必ずゼロとなる．

MSE だけを減らせば良いというわけではないことがわかる．

このことを一般に過学習 (overfitting) という．回帰変数を増やせば，すなわちモデルを複雑にすれば MSE をいくらでも下げることができる一つの例である．

過学習となっている場合は，学習データ近傍であっても回帰曲線は正しい推定値を与えない．推定誤差が低くなってしまう．このことを確かめるために同じ分布に従う別なデータセットを取って確かめてみよう．

In [ ]:
# テスト用 dataset
X2 = np.arange(0.1, 3, 0.3)
Y2 = A0 + A1*X2 + A2*X2**2 +0.5*np.random.randn(X2.size)

In [ ]:
# 多項式回帰とテストデータの比較
def plot_polynomialregression_fortestdata(M=2):
    # dataset
    X_train = np.zeros((N, M))

    reg = linear_model.LinearRegression()
    for j in range(M):
        X_train[:, j] = X**(j+1)

    reg.fit(X_train, Y)

    A_pred = np.zeros(M+1)
    A_pred[0] = reg.intercept_
    A_pred[1:M+1] = reg.coef_

    XX = np.arange(-0.2, 3, 0.01)
    YY = np.ones(XX.size)*A_pred[0]
    Y2_pred = np.ones(X2.size)*A_pred[0]
    for j in range(1, M+1):
        YY += A_pred[j] * XX**j
        Y2_pred += A_pred[j] * X2**j

    fig = plt.figure()
    ax = plt.axes()
    ax.set_xlabel("X", size=20)
    ax.set_ylabel("Y", size=20)
    ax.set_xticks
    ax.set_ylim(-5, 5)
    ax.scatter(X2, Y2, s=80)
    ax.plot(XX, YY, linewidth=3, c='tab:orange')
    ax.set_title('MSE = %f'%(np.sum((Y2-Y2_pred)**2)), size=20)
    ax.tick_params(labelsize=12)

In [ ]:
interact(plot_polynomialregression_fortestdata, M=(1, 20, 1))

テストデータに対する MSE は次数を上げるにつれて大きくなることがわかる．

学習で得られたモデルをテストデータに適用して得られた回帰誤差を汎化誤差と呼ぶ．

一方，学習時の回帰誤差を学習誤差と呼ぶ．

学習誤差は多項式の次数を上げることでいくらでも下げられるが，汎化誤差はそうとは限らない．汎化誤差も小さいモデルが良いモデルであると言える．

In [ ]:
Mmax = 15
MSE_train = np.zeros(Mmax+1)
MSE_test = np.zeros(Mmax+1)
for M in range(1, Mmax+1):
    # dataset
    X_train = np.zeros((X.size, M))

    reg = linear_model.LinearRegression()
    for j in range(M):
        X_train[:, j] = X**(j+1)

    reg.fit(X_train, Y)

    A_pred = np.zeros(M+1)
    A_pred[0] = reg.intercept_
    A_pred[1:M+1] = reg.coef_

    XX = np.arange(-0.2, 3, 0.01)
    YY = np.ones(XX.size)*A_pred[0]
    Y_pred = np.ones(X.size)*A_pred[0]
    Y2_pred = np.ones(X2.size)*A_pred[0]
    for j in range(1, M+1):
        YY += A_pred[j] * XX**j
        Y_pred += A_pred[j] * X**j
        Y2_pred += A_pred[j] * X2**j

    MSE_train[M] = np.sum((Y-Y_pred)**2)/Y.size
    MSE_test[M] = np.sum((Y2-Y2_pred)**2)/Y.size

# MSE のプロット
fig = plt.figure()
ax = plt.axes()
ax.set_xlabel("M: degree of polynomial", size=20)
ax.set_ylabel("$\log(MSE)$", size=20)
ax.set_ylim(-5, 6)
ax.set_xticks(np.arange(1, Mmax+1, 1))
ax.plot(np.arange(1, Mmax+1, 1), np.log10(MSE_train[1:]), 'o-', linewidth=3, c='tab:blue', label='training MSE')
ax.plot(np.arange(1, Mmax+1, 1), np.log10(MSE_test[1:]), 'o-', linewidth=3, c='tab:orange', label='test MSE')
ax.tick_params(labelsize=12)
ax.legend(fontsize=12)

汎化誤差が最小になる次数を取ることで，過学習を防ぐことができる．

# 演習問題 3-2

第3回その1で Iris データセットに対して，あらゆる変数の組み合わせについて線形回帰モデルを試した．

では，目的変数と説明変数を1つずつ取り，その組み合わせを取り替えて多項式回帰モデルを試し，多項式次数とフィッティング精度に傾向があるか確かめよ．上でやったように多項式次数を十分高く取ると過学習してしまうので，汎化誤差が小さくなるように多項式次数を調整し，学習誤差と汎化誤差の比較も行うこと．

（植物学的に意味があるかはさておき）一番フィッティングの合う多項式回帰モデルは何か．その理由も考えよ．

<h3><div style="text-align: right;">その3につづく</div></h3>